In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm $DL/*

In [3]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SHOP', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [5]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


In [6]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [7]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [8]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
AMD
AMZN


### E*Trade

In [15]:
DONGMEI = False

In [16]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
AAPL,5950.44
ABMD,2142.56
ABNB,1365.6
AMD,93.06
AMZN,3034.13


In [17]:
et_df.loc[et_df.ET == '--']
#et_df.loc['TASK', 'ET'] = 36.41

,ET


In [18]:
check_symbols(et_df)

All clear


In [19]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,5950
ABMD,2143
ABNB,1366
AMD,93
AMZN,3034


### Fidelity

In [20]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Apr-14-2022.csv'

In [21]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,2213.96,$1.00,$0.00,$2213.96,NaN,NaN,NaN,NaN,4.98%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,16.00,$165.29,-$5.11,$2644.64,-$81.76,-3.00%,+$220.48,+9.09%,5.95%,$2424.16,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,1.00,$306.08,+$1.65,$306.08,+$1.65,+0.54%,+$20.22,+7.07%,0.69%,$285.86,$285.86,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,3.00,$170.70,-$1.15,$512.10,-$3.45,-0.67%,+$83.85,+19.57%,1.15%,$428.25,$142.75,Cash
4,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,10.00,$93.06,-$4.68,$930.60,-$46.80,-4.79%,-$94.74,-9.24%,2.09%,$1025.34,$102.53,Cash


In [22]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,2645
2,ROLLOVER IRA,ABMD,306
3,ROLLOVER IRA,ABNB,512
4,ROLLOVER IRA,AMD,931
5,ROLLOVER IRA,ANET,1266


In [23]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,2645
ABMD,306
ABNB,512
AMD,931
ANET,1266


In [24]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [25]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,5950,2645,2645,0
ABMD,2143,306,1224,0
ABNB,1366,512,1024,0
AMD,93,931,5770,0
AMZN,3034,0,3034,0


In [26]:
out.to_csv(f'{DL}/clean.csv')

In [27]:
!open $DL/clean.csv

In [28]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
mw.csv


### MarketWatch

In [9]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv


In [10]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

Holdings - Damian Satterthwaite-Phillips(0).csv


,Value_0
AAPL,992
ABMD,918
ABNB,683
AMD,1303
AMZN,3034


In [11]:
check_symbols(mw)

All clear


In [12]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_0
AAPL,992
ABMD,918
ABNB,683
AMD,1303
AMZN,3034


In [13]:
mw.to_csv(f'{DL}/mw.csv')

In [14]:
!open $DL/mw.csv

In [29]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [16]:
!rm $DL/*

rm: /Users/damiansp/Downloads/*: No such file or directory


In [ ]:
!ls .

In [ ]:
!cp qp50.py.ipynb qp03.py.ipynb